In [ ]:
%%writefile scripts/dev-frontend.sh
#!/bin/bash

# Development script to run the React frontend
set -e

echo "🚀 Starting NFL Kicker Assessment Development Servers"
echo "=================================================="

# Change to the workspace directory
cd /workspace

# Check if we're in the container
if [ ! -d "/workspace/frontend" ] || [ ! -d "/workspace/backend" ]; then
    echo "❌ Error: frontend/ and backend/ directories not found"
    echo "   Make sure you're running this from within the dev container"
    exit 1
fi

# Function to cleanup background processes
cleanup() {
    echo "🛑 Shutting down development servers..."
    kill $(jobs -p) 2>/dev/null || true
    exit 0
}
trap cleanup SIGINT SIGTERM

# Install frontend dependencies if needed
echo "📦 Checking frontend dependencies..."
cd /workspace/frontend
if [ ! -d "node_modules" ]; then
    echo "   Installing frontend dependencies..."
    npm install
fi

# Install backend dependencies if needed
echo "📦 Checking backend dependencies..."
cd /workspace/backend
if [ ! -d "node_modules" ]; then
    echo "   Installing backend dependencies..."
    npm install
fi

# Start backend development server
echo "🔧 Starting backend development server on port 5000..."
cd /workspace/backend
npm run dev &
BACKEND_PID=$!

# Wait a moment for backend to start
sleep 3

# Start frontend development server
echo "🎨 Starting frontend development server on port 5173..."
cd /workspace/frontend
npm run dev &
FRONTEND_PID=$!

echo ""
echo "✅ Development servers started!"
echo "📊 Frontend: http://localhost:5173"
echo "🔧 Backend API: http://localhost:5002 (proxied from frontend)"
echo "🏥 Health check: http://localhost:5002/api/ping"
echo ""
echo "Press Ctrl+C to stop both servers"

# Wait for both processes
wait $BACKEND_PID $FRONTEND_PID 


In [ ]:
%%writefile Dockerfile
# Multi-stage Dockerfile for React + Express deployment
# Stage 1: Build the React frontend
FROM node:18-alpine AS client-build
WORKDIR /client
COPY frontend/package*.json ./
RUN npm ci
COPY frontend/. .
RUN npm run build                   # output will be /client/dist

# Stage 2: Setup Express backend and serve static files
FROM node:18-alpine

# Create non-root user for security
RUN addgroup -g 1001 -S nodejs && \
    adduser -S nextjs -u 1001

WORKDIR /app

# Copy backend package files and install (prod only)
COPY backend/package*.json ./
RUN npm ci --omit=dev

# Copy backend source code
COPY backend/. .

# Copy frontend build artifacts into backend's public/ folder
COPY --from=client-build /client/dist ./public

# Set proper ownership for non-root user
RUN chown -R nextjs:nodejs /app
USER nextjs

# Set environment and port - Railway injects PORT at runtime
ENV NODE_ENV=production
ENV PORT=${PORT:-5000}
EXPOSE $PORT

# Add health check for Railway deployment health monitoring
HEALTHCHECK --interval=30s --timeout=10s --start-period=5s --retries=3 \
  CMD node -e "require('http').get('http://localhost:' + process.env.PORT + '/api/ping', (res) => { process.exit(res.statusCode === 200 ? 0 : 1) }).on('error', () => process.exit(1))"

# Launch the Express server (which serves static files from /app/public)
CMD ["node", "server.js"] 


In [ ]:
%%writefile backend/server.js
import express from 'express';
import cors from 'cors';
import multer from 'multer';
import csv from 'csv-parser';
import fs from 'fs';
import path from 'path';
import { fileURLToPath } from 'url';

const __filename = fileURLToPath(import.meta.url);
const __dirname = path.dirname(__filename);

const app = express();

// Middleware
app.use(cors({
  origin: process.env.CORS_ORIGIN 
    ? process.env.CORS_ORIGIN.split(',').map(origin => origin.trim())
    : process.env.NODE_ENV === 'production' 
      ? ['https://your-railway-domain.com'] // Will be replaced with actual Railway domain
      : ['http://localhost:5173', 'http://localhost:3000'],
  credentials: true
}));

app.use(express.json());
app.use(express.urlencoded({ extended: true }));

// In production, serve static files from the public directory
if (process.env.NODE_ENV === 'production') {
  const publicPath = path.join(__dirname, 'public');
  app.use(express.static(publicPath));
  
  // Serve index.html for any non-API routes (SPA fallback)
  app.get('*', (req, res, next) => {
    if (req.path.startsWith('/api/')) {
      return next();
    }
    res.sendFile(path.join(publicPath, 'index.html'));
  });
}

// Configure multer for file uploads
const storage = multer.diskStorage({
  destination: (req, file, cb) => {
    const uploadDir = 'uploads/';
    // Ensure uploads directory exists (Railway volume mount point)
    if (!fs.existsSync(uploadDir)) {
      fs.mkdirSync(uploadDir, { recursive: true });
    }
    cb(null, uploadDir);
  },
  filename: (req, file, cb) => {
    // Add timestamp to prevent filename conflicts
    const timestamp = Date.now();
    const originalName = file.originalname;
    cb(null, `${timestamp}-${originalName}`);
  }
});

const upload = multer({ 
  storage,
  limits: {
    fileSize: 10 * 1024 * 1024, // 10MB limit
    files: 1
  },
  fileFilter: (req, file, cb) => {
    // Only allow CSV files
    if (file.mimetype === 'text/csv' || file.originalname.endsWith('.csv')) {
      cb(null, true);
    } else {
      cb(new Error('Only CSV files are allowed'), false);
    }
  }
});

// API Routes

// Health check endpoint
app.get('/api/ping', (req, res) => {
  res.json({ 
    pong: true, 
    timestamp: new Date().toISOString(),
    environment: process.env.NODE_ENV || 'development'
  });
});

// CSV upload and processing endpoint
app.post('/api/upload', upload.single('file'), (req, res) => {
  if (!req.file) {
    return res.status(400).json({ error: 'No file uploaded' });
  }

  const results = [];
  const filePath = req.file.path;

  console.log(`Processing uploaded file: ${req.file.originalname}`);

  // Stream parse the CSV file
  fs.createReadStream(filePath)
    .pipe(csv())
    .on('data', (data) => {
      // Clean and validate data
      const cleanData = {};
      Object.keys(data).forEach(key => {
        const cleanKey = key.trim();
        const value = data[key]?.toString().trim();
        
        // Convert numeric fields
        if (['player_id', 'distance', 'made', 'week', 'year'].includes(cleanKey)) {
          cleanData[cleanKey] = isNaN(value) ? value : Number(value);
        } else {
          cleanData[cleanKey] = value;
        }
      });
      
      // Basic validation - ensure required fields are present
      if (cleanData.player_id && cleanData.player_name && 
          cleanData.distance !== undefined && cleanData.made !== undefined) {
        results.push(cleanData);
      }
    })
    .on('end', () => {
      // Clean up the uploaded file
      fs.unlink(filePath, (err) => {
        if (err) console.error('Error deleting uploaded file:', err);
      });

      console.log(`Processed ${results.length} records from ${req.file.originalname}`);
      
      res.json({ 
        success: true,
        data: results,
        rowCount: results.length,
        filename: req.file.originalname
      });
    })
    .on('error', (error) => {
      console.error('Error processing CSV:', error);
      
      // Clean up the uploaded file on error
      fs.unlink(filePath, (err) => {
        if (err) console.error('Error deleting uploaded file:', err);
      });
      
      res.status(500).json({ 
        error: 'Error processing CSV file',
        details: error.message
      });
    });
});

// Get processed leaderboard data (if we want to store it server-side)
app.get('/api/leaderboard', (req, res) => {
  // This could be enhanced to return cached/processed leaderboard data
  res.json({ 
    message: 'Leaderboard endpoint - not implemented yet',
    timestamp: new Date().toISOString()
  });
});

// Error handling middleware
app.use((error, req, res, next) => {
  console.error('Server error:', error);
  
  if (error instanceof multer.MulterError) {
    if (error.code === 'LIMIT_FILE_SIZE') {
      return res.status(400).json({ error: 'File too large (max 10MB)' });
    }
    if (error.code === 'LIMIT_UNEXPECTED_FILE') {
      return res.status(400).json({ error: 'Too many files uploaded' });
    }
  }
  
  res.status(500).json({ 
    error: 'Internal server error',
    details: process.env.NODE_ENV === 'development' ? error.message : undefined
  });
});

// 404 handler for API routes
app.use('/api/*', (req, res) => {
  res.status(404).json({ error: 'API endpoint not found' });
});

// Start server
const PORT = process.env.PORT || 5000;
const HOST = process.env.HOST || '0.0.0.0';

app.listen(PORT, HOST, () => {
  console.log(`🚀 NFL Kicker API server running on http://${HOST}:${PORT}`);
  console.log(`📁 Environment: ${process.env.NODE_ENV || 'development'}`);
  console.log(`🔧 CORS enabled for: ${process.env.CORS_ORIGIN || (process.env.NODE_ENV === 'production' ? 'production domains' : 'development (localhost:5173, localhost:3000)')}`);
  console.log(`📊 Health check: http://${HOST}:${PORT}/api/ping`);
  
  // Railway deployment info
  if (process.env.RAILWAY_ENVIRONMENT) {
    console.log(`🚂 Railway Environment: ${process.env.RAILWAY_ENVIRONMENT}`);
    console.log(`🌍 Railway Service: ${process.env.RAILWAY_SERVICE_NAME || 'Unknown'}`);
  }
});

// Graceful shutdown
process.on('SIGTERM', () => {
  console.log('SIGTERM received, shutting down gracefully');
  process.exit(0);
});

process.on('SIGINT', () => {
  console.log('SIGINT received, shutting down gracefully');
  process.exit(0);
}); 



In [ ]:
%%writefile backend/package.json
{
  "name": "nfl-kicker-backend",
  "version": "1.0.0",
  "description": "Express backend for NFL Kicker Assessment",
  "main": "server.js",
  "type": "module",
  "scripts": {
    "start": "node server.js",
    "dev": "nodemon server.js"
  },
  "dependencies": {
    "express": "^4.18.2",
    "cors": "^2.8.5",
    "multer": "^1.4.5-lts.1",
    "csv-parser": "^3.0.0"
  },
  "devDependencies": {
    "nodemon": "^3.0.2"
  },
  "keywords": ["nfl", "kicker", "assessment", "api"],
  "author": "",
  "license": "MIT"
} 


In [ ]:
%%writefile frontend/index.html
<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <link rel="icon" type="image/svg+xml" href="/vite.svg" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>NFL Kicker Assessment</title>
  </head>
  <body>
    <div id="root"></div>
    <script type="module" src="/src/main.jsx"></script>
  </body>
</html> 

In [ ]:
%%writefile frontend/postcss.config.js
export default {
  plugins: {
    tailwindcss: {},
    autoprefixer: {},
  },
} 

In [ ]:
%%writefile frontend/tailwind.config.js
/** @type {import('tailwindcss').Config} */
export default {
  content: [
    "./index.html",
    "./src/**/*.{js,ts,jsx,tsx}",
  ],
  theme: {
    extend: {},
  },
  plugins: [],
} 

In [ ]:
%%writefile frontend/vite.config.js
import { defineConfig } from 'vite'
import react from '@vitejs/plugin-react'

// https://vitejs.dev/config/
export default defineConfig({
  plugins: [react()],
  server: {
    host: '0.0.0.0',
    port: 5173,
    proxy: {
      '/api': {
        target: 'http://localhost:5000',
        changeOrigin: true,
        secure: false,
      }
    }
  }
}) 

In [ ]:
%%writefile frontend/package.json
{
  "name": "nfl-kicker-frontend",
  "private": true,
  "version": "0.0.0",
  "type": "module",
  "scripts": {
    "dev": "vite",
    "build": "vite build",
    "start": "serve -s dist -l $PORT",
    "lint": "eslint . --ext js,jsx --report-unused-disable-directives --max-warnings 0",
    "preview": "vite preview"
  },
  "dependencies": {
    "react": "^18.2.0",
    "react-dom": "^18.2.0",
    "recharts": "^2.12.7",
    "lucide-react": "^0.436.0"
  },
  "devDependencies": {
    "@types/react": "^18.2.43",
    "@types/react-dom": "^18.2.17",
    "@vitejs/plugin-react": "^4.2.1",
    "autoprefixer": "^10.4.16",
    "eslint": "^8.55.0",
    "eslint-plugin-react": "^7.33.2",
    "eslint-plugin-react-hooks": "^4.6.0",
    "eslint-plugin-react-refresh": "^0.4.5",
    "postcss": "^8.4.32",
    "tailwindcss": "^3.3.6",
    "vite": "^5.0.8"
  }
} 

In [ ]:
%%writefile frontend/src/index.css
@tailwind base;
@tailwind components;
@tailwind utilities; 

In [ ]:
%%writefile frontend/src/main.jsx
import React from 'react'
import ReactDOM from 'react-dom/client'
import App from './App.jsx'
import './index.css'

ReactDOM.createRoot(document.getElementById('root')).render(
  <React.StrictMode>
    <App />
  </React.StrictMode>,
) 

In [ ]:
%%writefile frontend/src/App.jsx

import React, { useState, useEffect } from 'react';
import { Upload, Download, BarChart3, Users, Target, TrendingUp, FileText, Calculator } from 'lucide-react';
import { LineChart, Line, XAxis, YAxis, CartesianGrid, Tooltip, Legend, ScatterChart, Scatter, ResponsiveContainer, BarChart, Bar } from 'recharts';

const NFLKickerAssessment = () => {
  const [data, setData] = useState(null);
  const [processedData, setProcessedData] = useState(null);
  const [leaderboard, setLeaderboard] = useState([]);
  const [activeTab, setActiveTab] = useState('upload');
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState(null);
  const [modelParams, setModelParams] = useState({
    minAttempts: 10,
    weightRecency: 0.3,
    weightDistance: 0.4,
    weightAccuracy: 0.3
  });

  const calculateKickerRating = (kickerData) => {
    if (!kickerData || kickerData.length === 0) return 0;
    
    // Calculate basic statistics
    const totalAttempts = kickerData.length;
    const totalMakes = kickerData.filter(kick => kick.made === 1).length;
    const accuracy = totalMakes / totalAttempts;
    
    // Distance-weighted accuracy
    const distanceWeightedScore = kickerData.reduce((sum, kick) => {
      const distanceWeight = Math.max(0, (kick.distance - 20) / 40); // Normalize distance
      return sum + (kick.made * (1 + distanceWeight));
    }, 0) / totalAttempts;
    
    // Recency weight (more recent kicks weighted higher)
    const maxWeek = Math.max(...kickerData.map(k => k.week));
    const recencyScore = kickerData.reduce((sum, kick) => {
      const recencyWeight = kick.week / maxWeek;
      return sum + (kick.made * recencyWeight);
    }, 0) / totalAttempts;
    
    // Composite rating
    const rating = (
      modelParams.weightAccuracy * accuracy +
      modelParams.weightDistance * distanceWeightedScore +
      modelParams.weightRecency * recencyScore
    ) * 100;
    
    return Math.round(rating * 100) / 100;
  };

  const processData = () => {
    if (!data) return;
    
    setLoading(true);
    try {
      // Group by player
      const playerGroups = {};
      data.forEach(row => {
        if (!playerGroups[row.player_id]) {
          playerGroups[row.player_id] = {
            player_id: row.player_id,
            player_name: row.player_name,
            attempts: []
          };
        }
        playerGroups[row.player_id].attempts.push(row);
      });
      
      // Calculate ratings and create leaderboard
      const leaderboardData = Object.values(playerGroups)
        .filter(player => player.attempts.length >= modelParams.minAttempts)
        .map(player => ({
          player_id: player.player_id,
          player_name: player.player_name,
          rating: calculateKickerRating(player.attempts),
          attempts: player.attempts.length,
          accuracy: player.attempts.filter(a => a.made === 1).length / player.attempts.length
        }))
        .sort((a, b) => b.rating - a.rating)
        .map((player, index) => ({ ...player, rank: index + 1 }));
      
      setLeaderboard(leaderboardData);
      setProcessedData(playerGroups);
    } catch (err) {
      setError('Error processing data: ' + err.message);
    } finally {
      setLoading(false);
    }
  };

  const handleFileUpload = async (event) => {
    const file = event.target.files[0];
    if (!file) return;

    if (file.type !== 'text/csv') {
      setError('Please upload a CSV file');
      return;
    }

    setLoading(true);
    setError(null);

    try {
      // Try backend API first
      const formData = new FormData();
      formData.append('file', file);
      
      const response = await fetch('/api/upload', {
        method: 'POST',
        body: formData,
      });

      if (response.ok) {
        const result = await response.json();
        setData(result.data);
      } else {
        throw new Error('Backend API not available');
      }
      
    } catch (err) {
      // Fallback to client-side processing
      console.log('Using client-side processing:', err.message);
      
      const reader = new FileReader();
      reader.onload = (e) => {
        try {
          const text = e.target.result;
          const rows = text.split('\n').filter(row => row.trim());
          const headers = rows[0].split(',').map(h => h.trim());
          const parsedData = rows.slice(1).map(row => {
            const values = row.split(',');
            const obj = {};
            headers.forEach((header, index) => {
              const value = values[index]?.trim();
              obj[header] = isNaN(value) ? value : Number(value);
            });
            return obj;
          });
          setData(parsedData);
          setError(null);
        } catch (parseErr) {
          setError('Error parsing CSV file: ' + parseErr.message);
        }
      };
      reader.readAsText(file);
    } finally {
      setLoading(false);
    }
  };

  const downloadLeaderboard = () => {
    const csv = [
      'player_id,player_name,rating,rank',
      ...leaderboard.map(row => `${row.player_id},${row.player_name},${row.rating},${row.rank}`)
    ].join('\n');
    
    const blob = new Blob([csv], { type: 'text/csv' });
    const url = window.URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.style.display = 'none';
    a.href = url;
    a.download = 'leaderboard.csv';
    document.body.appendChild(a);
    a.click();
    window.URL.revokeObjectURL(url);
  };

  const generateReport = () => {
    const report = `# NFL Kicker Rating Model - Technical Assessment

## Model Overview
This rating system evaluates NFL kickers based on:
- **Accuracy**: Base field goal percentage
- **Distance Performance**: Weighted by kick difficulty (distance)
- **Recency**: More recent performance weighted higher

## Model Parameters
- Minimum Attempts: ${modelParams.minAttempts}
- Accuracy Weight: ${modelParams.weightAccuracy}
- Distance Weight: ${modelParams.weightDistance}
- Recency Weight: ${modelParams.weightRecency}

## Top 10 Kickers (Week 6, 2018)
${leaderboard.slice(0, 10).map((k, i) => 
  `${i + 1}. ${k.player_name} - Rating: ${k.rating} (${k.attempts} attempts, ${(k.accuracy * 100).toFixed(1)}% accuracy)`
).join('\n')}

## Model Interpretation
- Rating Scale: 0-100 (higher is better)
- Considers both volume and efficiency
- Adjusts for kick difficulty and timing

## Potential Improvements
1. Incorporate situational factors (weather, pressure)
2. Add opponent strength adjustments
3. Include career trajectory analysis
4. Implement Bayesian updating for new kickers

## Next Steps
1. Validate model with out-of-sample data
2. Compare against Vegas odds or expert rankings
3. Implement confidence intervals
4. Add visualization for decision-makers
    `;
    
    const blob = new Blob([report], { type: 'text/markdown' });
    const url = window.URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.style.display = 'none';
    a.href = url;
    a.download = 'kicker_analysis_report.md';
    document.body.appendChild(a);
    a.click();
    window.URL.revokeObjectURL(url);
  };

  useEffect(() => {
    if (data) {
      processData();
    }
  }, [data, modelParams]);

  const TabButton = ({ id, label, icon: Icon, active }) => (
    <button
      onClick={() => setActiveTab(id)}
      className={`flex items-center px-4 py-2 rounded-lg transition-colors ${
        active 
          ? 'bg-orange-600 text-white' 
          : 'bg-gray-100 text-gray-700 hover:bg-gray-200'
      }`}
    >
      <Icon className="w-4 h-4 mr-2" />
      {label}
    </button>
  );

  return (
    <div className="min-h-screen bg-gradient-to-br from-orange-50 to-blue-50 p-6">
      <div className="max-w-7xl mx-auto">
        <div className="bg-white rounded-xl shadow-lg overflow-hidden">
          {/* Header */}
          <div className="bg-gradient-to-r from-orange-600 to-blue-600 text-white p-6">
            <h1 className="text-3xl font-bold flex items-center">
              <Target className="w-8 h-8 mr-3" />
              NFL Kicker Assessment Framework
            </h1>
            <p className="text-orange-100 mt-2">Denver Broncos Technical Assessment Tool</p>
          </div>

          {/* Navigation */}
          <div className="border-b border-gray-200 p-4">
            <div className="flex flex-wrap gap-2">
              <TabButton id="upload" label="Data Upload" icon={Upload} active={activeTab === 'upload'} />
              <TabButton id="model" label="Model Config" icon={Calculator} active={activeTab === 'model'} />
              <TabButton id="leaderboard" label="Leaderboard" icon={Users} active={activeTab === 'leaderboard'} />
              <TabButton id="analysis" label="Analysis" icon={BarChart3} active={activeTab === 'analysis'} />
              <TabButton id="export" label="Export" icon={Download} active={activeTab === 'export'} />
            </div>
          </div>

          {/* Content */}
          <div className="p-6">
            {/* Error Display */}
            {error && (
              <div className="bg-red-50 border border-red-200 rounded-lg p-4 mb-6">
                <h4 className="font-semibold text-red-800">Error</h4>
                <p className="text-red-700">{error}</p>
              </div>
            )}

            {activeTab === 'upload' && (
              <div className="space-y-6">
                <div className="border-2 border-dashed border-gray-300 rounded-lg p-8 text-center">
                  <Upload className="w-12 h-12 mx-auto text-gray-400 mb-4" />
                  <h3 className="text-lg font-semibold mb-2">Upload NFL Field Goal Data</h3>
                  <p className="text-gray-600 mb-4">Upload the CSV file with field goal data</p>
                  <input
                    type="file"
                    accept=".csv"
                    onChange={handleFileUpload}
                    disabled={loading}
                    className="block w-full text-sm text-gray-500 file:mr-4 file:py-2 file:px-4 file:rounded-full file:border-0 file:text-sm file:font-semibold file:bg-orange-50 file:text-orange-700 hover:file:bg-orange-100 disabled:opacity-50"
                  />
                  {loading && (
                    <div className="mt-4">
                      <div className="animate-spin rounded-full h-8 w-8 border-b-2 border-orange-600 mx-auto"></div>
                      <p className="text-gray-600 mt-2">Processing file...</p>
                    </div>
                  )}
                </div>
                
                {data && (
                  <div className="bg-green-50 border border-green-200 rounded-lg p-4">
                    <h4 className="font-semibold text-green-800">Data Loaded Successfully!</h4>
                    <p className="text-green-700">Loaded {data.length} field goal attempts</p>
                    <div className="mt-2 text-sm">
                      <strong>Sample data structure:</strong>
                      <pre className="bg-white p-2 rounded mt-1 text-xs overflow-x-auto">
                        {JSON.stringify(data[0], null, 2)}
                      </pre>
                    </div>
                  </div>
                )}
              </div>
            )}

            {activeTab === 'model' && (
              <div className="space-y-6">
                <h3 className="text-xl font-bold">Model Configuration</h3>
                
                <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
                  <div className="space-y-4">
                    <div>
                      <label className="block text-sm font-medium mb-2">Minimum Attempts</label>
                      <input
                        type="number"
                        value={modelParams.minAttempts}
                        onChange={(e) => setModelParams({...modelParams, minAttempts: parseInt(e.target.value)})}
                        className="w-full px-3 py-2 border rounded-lg focus:ring-2 focus:ring-orange-500 focus:border-transparent"
                      />
                    </div>
                    
                    <div>
                      <label className="block text-sm font-medium mb-2">Accuracy Weight</label>
                      <input
                        type="range"
                        min="0"
                        max="1"
                        step="0.1"
                        value={modelParams.weightAccuracy}
                        onChange={(e) => setModelParams({...modelParams, weightAccuracy: parseFloat(e.target.value)})}
                        className="w-full"
                      />
                      <span className="text-sm text-gray-600">{modelParams.weightAccuracy}</span>
                    </div>
                    
                    <div>
                      <label className="block text-sm font-medium mb-2">Distance Weight</label>
                      <input
                        type="range"
                        min="0"
                        max="1"
                        step="0.1"
                        value={modelParams.weightDistance}
                        onChange={(e) => setModelParams({...modelParams, weightDistance: parseFloat(e.target.value)})}
                        className="w-full"
                      />
                      <span className="text-sm text-gray-600">{modelParams.weightDistance}</span>
                    </div>
                    
                    <div>
                      <label className="block text-sm font-medium mb-2">Recency Weight</label>
                      <input
                        type="range"
                        min="0"
                        max="1"
                        step="0.1"
                        value={modelParams.weightRecency}
                        onChange={(e) => setModelParams({...modelParams, weightRecency: parseFloat(e.target.value)})}
                        className="w-full"
                      />
                      <span className="text-sm text-gray-600">{modelParams.weightRecency}</span>
                    </div>
                  </div>
                  
                  <div className="bg-gray-50 p-4 rounded-lg">
                    <h4 className="font-semibold mb-3">Model Explanation</h4>
                    <div className="space-y-2 text-sm">
                      <p><strong>Accuracy:</strong> Base field goal percentage</p>
                      <p><strong>Distance:</strong> Rewards longer successful kicks</p>
                      <p><strong>Recency:</strong> Weights recent performance higher</p>
                      <p className="text-gray-600 text-xs mt-3">
                        The final rating is a weighted combination of these factors, 
                        scaled to 0-100 where higher is better.
                      </p>
                    </div>
                  </div>
                </div>
              </div>
            )}

            {activeTab === 'leaderboard' && (
              <div className="space-y-6">
                <div className="flex justify-between items-center">
                  <h3 className="text-xl font-bold">Kicker Leaderboard</h3>
                  {leaderboard.length > 0 && (
                    <button
                      onClick={downloadLeaderboard}
                      className="bg-orange-600 text-white px-4 py-2 rounded-lg hover:bg-orange-700 flex items-center transition-colors"
                    >
                      <Download className="w-4 h-4 mr-2" />
                      Download CSV
                    </button>
                  )}
                </div>
                
                {leaderboard.length > 0 ? (
                  <div className="overflow-x-auto">
                    <table className="w-full border-collapse border border-gray-300">
                      <thead className="bg-gray-50">
                        <tr>
                          <th className="border border-gray-300 px-4 py-2 text-left">Rank</th>
                          <th className="border border-gray-300 px-4 py-2 text-left">Player Name</th>
                          <th className="border border-gray-300 px-4 py-2 text-left">Rating</th>
                          <th className="border border-gray-300 px-4 py-2 text-left">Attempts</th>
                          <th className="border border-gray-300 px-4 py-2 text-left">Accuracy</th>
                        </tr>
                      </thead>
                      <tbody>
                        {leaderboard.map(kicker => (
                          <tr key={kicker.player_id} className="hover:bg-gray-50">
                            <td className="border border-gray-300 px-4 py-2">{kicker.rank}</td>
                            <td className="border border-gray-300 px-4 py-2 font-medium">{kicker.player_name}</td>
                            <td className="border border-gray-300 px-4 py-2">{kicker.rating}</td>
                            <td className="border border-gray-300 px-4 py-2">{kicker.attempts}</td>
                            <td className="border border-gray-300 px-4 py-2">{(kicker.accuracy * 100).toFixed(1)}%</td>
                          </tr>
                        ))}
                      </tbody>
                    </table>
                  </div>
                ) : (
                  <div className="text-center py-12 text-gray-500">
                    Upload data and configure the model to see the leaderboard
                  </div>
                )}
              </div>
            )}

            {activeTab === 'analysis' && (
              <div className="space-y-6">
                <h3 className="text-xl font-bold">Data Analysis</h3>
                
                {leaderboard.length > 0 ? (
                  <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
                    <div className="bg-white border rounded-lg p-4">
                      <h4 className="font-semibold mb-3">Rating Distribution</h4>
                      <ResponsiveContainer width="100%" height={300}>
                        <BarChart data={leaderboard.slice(0, 10)}>
                          <CartesianGrid strokeDasharray="3 3" />
                          <XAxis 
                            dataKey="player_name" 
                            angle={-45}
                            textAnchor="end"
                            height={100}
                          />
                          <YAxis />
                          <Tooltip />
                          <Bar dataKey="rating" fill="#ea580c" />
                        </BarChart>
                      </ResponsiveContainer>
                    </div>
                    
                    <div className="bg-white border rounded-lg p-4">
                      <h4 className="font-semibold mb-3">Rating vs Accuracy</h4>
                      <ResponsiveContainer width="100%" height={300}>
                        <ScatterChart data={leaderboard}>
                          <CartesianGrid strokeDasharray="3 3" />
                          <XAxis dataKey="accuracy" domain={[0, 1]} />
                          <YAxis dataKey="rating" />
                          <Tooltip 
                            formatter={(value, name) => [
                              name === 'accuracy' ? `${(value * 100).toFixed(1)}%` : value,
                              name === 'accuracy' ? 'Accuracy' : 'Rating'
                            ]}
                          />
                          <Scatter dataKey="rating" fill="#2563eb" />
                        </ScatterChart>
                      </ResponsiveContainer>
                    </div>
                  </div>
                ) : (
                  <div className="text-center py-12 text-gray-500">
                    Upload data to see analysis charts
                  </div>
                )}
              </div>
            )}

            {activeTab === 'export' && (
              <div className="space-y-6">
                <h3 className="text-xl font-bold">Export Deliverables</h3>
                
                <div className="grid grid-cols-1 md:grid-cols-3 gap-4">
                  <div className="bg-blue-50 border border-blue-200 rounded-lg p-6">
                    <FileText className="w-8 h-8 text-blue-600 mb-3" />
                    <h4 className="font-semibold mb-2">Leaderboard CSV</h4>
                    <p className="text-sm text-gray-600 mb-4">Required format with player_id, player_name, rating, rank</p>
                    <button
                      onClick={downloadLeaderboard}
                      disabled={leaderboard.length === 0}
                      className="w-full bg-blue-600 text-white px-4 py-2 rounded hover:bg-blue-700 disabled:bg-gray-300 transition-colors"
                    >
                      Download CSV
                    </button>
                  </div>
                  
                  <div className="bg-green-50 border border-green-200 rounded-lg p-6">
                    <TrendingUp className="w-8 h-8 text-green-600 mb-3" />
                    <h4 className="font-semibold mb-2">Analysis Report</h4>
                    <p className="text-sm text-gray-600 mb-4">Model explanation and critique document</p>
                    <button
                      onClick={generateReport}
                      disabled={leaderboard.length === 0}
                      className="w-full bg-green-600 text-white px-4 py-2 rounded hover:bg-green-700 disabled:bg-gray-300 transition-colors"
                    >
                      Generate Report
                    </button>
                  </div>
                  
                  <div className="bg-orange-50 border border-orange-200 rounded-lg p-6">
                    <Calculator className="w-8 h-8 text-orange-600 mb-3" />
                    <h4 className="font-semibold mb-2">Python Script</h4>
                    <p className="text-sm text-gray-600 mb-4">Documented code for model implementation</p>
                    <button
                      onClick={() => alert('Generate Python script based on your model configuration')}
                      className="w-full bg-orange-600 text-white px-4 py-2 rounded hover:bg-orange-700 transition-colors"
                    >
                      View Script
                    </button>
                  </div>
                </div>
                
                <div className="bg-gray-50 rounded-lg p-6">
                  <h4 className="font-semibold mb-3">Submission Checklist</h4>
                  <div className="space-y-2">
                    <label className="flex items-center">
                      <input type="checkbox" className="mr-2" />
                      <span className="text-sm">leaderboard.csv with required columns</span>
                    </label>
                    <label className="flex items-center">
                      <input type="checkbox" className="mr-2" />
                      <span className="text-sm">Well-documented Python/R script</span>
                    </label>
                    <label className="flex items-center">
                      <input type="checkbox" className="mr-2" />
                      <span className="text-sm">Model critique and improvement suggestions</span>
                    </label>
                    <label className="flex items-center">
                      <input type="checkbox" className="mr-2" />
                      <span className="text-sm">Email to footballresearch.technology@broncos.nfl.net</span>
                    </label>
                  </div>
                </div>
              </div>
            )}
          </div>
        </div>
      </div>
    </div>
  );
};

export default NFLKickerAssessment; 

In [ ]:
%%writefile railway.toml
[build]
# Railway automatically detects a Dockerfile, but we name the image for clarity
builder = "dockerfile"
dockerfilePath = "Dockerfile"

[deploy]
startCommand = "node server.js"
healthcheckPath = "/api/ping"
restartPolicyType = "on_failure"

[service]
# Expose your unified API + static bundle
ports = ["$PORT"]

[volumes]
# Persist model artifacts & uploads across deploys
"/app/uploads" = "kicker-uploads"
"/workspace/mlruns" = "mlflow-artifacts"
"/mlflow_db" = "mlflow-database" 

In [ ]:
%%writefile .devcontainer/generate-project-name.sh
#!/usr/bin/env bash
set -euo pipefail

ENV_FILE=".devcontainer/.env.runtime"

# Only generate once per local checkout
if [[ ! -f "${ENV_FILE}" ]]; then
  # workspaceFolderBasename → "docker-dev-template", devcontainerId → random but stable
  RAND="${devcontainerId:-$(date +%s | tail -c 6 | tr -d '\n')}"
  BASENAME="${localWorkspaceFolderBasename:-$(basename "$PWD")}"
  
  # Clean up basename to be docker-compose friendly
  BASENAME=$(echo "$BASENAME" | sed 's/[^a-zA-Z0-9_-]/-/g' | tr '[:upper:]' '[:lower:]')
  
  PROJECT_NAME="${BASENAME}-${RAND}"
  
  echo "COMPOSE_PROJECT_NAME=${PROJECT_NAME}" > "${ENV_FILE}"
  echo "ENV_NAME=${PROJECT_NAME}"           >> "${ENV_FILE}"
  echo "🔧  Generated unique project name: ${PROJECT_NAME}"
  echo "🔧  Wrote ${ENV_FILE}: $(cat ${ENV_FILE})"
fi 